In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
import transformers
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import TrainingArguments
from transformers import Trainer
import warnings
warnings.filterwarnings('ignore')

/home/payam/miniconda3/envs/nlpdev/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load the dataset

In [2]:
df = pd.read_csv(
    'https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv',
    delimiter='\t',
    header=None
)

In [3]:
df.shape

(6920, 2)

In [4]:
df.head()

,0,1
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1


In [5]:
# rename the columns
df.rename(
    columns={
        0: "text",
        1: "label"
    },
    inplace= True
)

In [6]:
df.head()

,text,label
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1


In [7]:
df["label"].value_counts()

label
1    3610
0    3310
Name: count, dtype: int64

# Load pretrained model/tokenizer

In [8]:
# DistilBERT:
# Load pretrained model/tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
tokenizer

DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [10]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [11]:
tokenizer.decode(tokenizer(df.iloc[0,0])["input_ids"])

'[CLS] a stirring, funny and finally transporting re imagining of beauty and the beast and 1930s horror films [SEP]'

In [12]:
tokenizer.decode(tokenizer("hey!, How are you?")["input_ids"])

'[CLS] hey!, how are you? [SEP]'

# Split data into train, test and validation sets

In [13]:
train_df, temp_df = train_test_split(df, test_size=0.4, random_state=42)  # 60% train, 40% temp
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)  # 20% validation, 20% test

print(f"Training Set: {len(train_df)} samples")
print(f"Validation Set: {len(val_df)} samples")
print(f"Test Set: {len(test_df)} samples")


Training Set: 4152 samples
Validation Set: 1384 samples
Test Set: 1384 samples


# Tokenizing

In [14]:
# Tokenize the text data
def tokenize_function(texts):
    return tokenizer(texts, padding= "longest", truncation=True, return_tensors="pt")

In [15]:
# train
train_texts  = train_df["text"].tolist()
train_labels = train_df["label"].tolist()
# validation
val_texts = val_df["text"].tolist()
val_labels =  val_df["label"].tolist()
#test
test_texts =  test_df["text"].tolist()
test_labels= test_df["label"].tolist()

# Create tokens
train_encodings = tokenize_function(train_texts)
val_encodings = tokenize_function(val_texts)
test_encodings = tokenize_function(test_texts)


In [16]:
train_encodings

{'input_ids': tensor([[  101,  3291,  2003,  ...,     0,     0,     0],
        [  101, 17688,  2094,  ..., 25269,  2497,   102],
        [  101,  3061,  1999,  ...,     0,     0,     0],
        ...,
        [  101,  2028,  1997,  ...,     0,     0,     0],
        [  101,  1996, 11669,  ...,     0,     0,     0],
        [  101,  1996,  2035,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

# Preparing data for Trainer

In [17]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [18]:
train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)
test_dataset = SentimentDataset(test_encodings, test_labels)

In [19]:
train_dataset

# Training (fine tuning)

In [20]:
training_args = TrainingArguments(
    output_dir="test_trainer",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    seed=42
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

In [21]:
trainer.train()


                                                 
 34%|███▎      | 262/780 [00:14<01:20,  6.45it/s]

{'eval_loss': 0.2789078652858734, 'eval_runtime': 1.0818, 'eval_samples_per_second': 1279.294, 'eval_steps_per_second': 80.418, 'epoch': 1.0}


 64%|██████▍   | 500/780 [00:26<00:13, 20.78it/s]

{'loss': 0.2907, 'grad_norm': 5.429376602172852, 'learning_rate': 1.794871794871795e-05, 'epoch': 1.92}


                                                 
 67%|██████▋   | 523/780 [00:29<00:43,  5.92it/s]

{'eval_loss': 0.38285043835639954, 'eval_runtime': 1.1014, 'eval_samples_per_second': 1256.579, 'eval_steps_per_second': 78.99, 'epoch': 2.0}


                                                 
100%|██████████| 780/780 [00:45<00:00, 17.30it/s]

{'eval_loss': 0.4941558539867401, 'eval_runtime': 1.1224, 'eval_samples_per_second': 1233.082, 'eval_steps_per_second': 77.513, 'epoch': 3.0}
{'train_runtime': 45.0848, 'train_samples_per_second': 276.279, 'train_steps_per_second': 17.301, 'train_loss': 0.21081879689143254, 'epoch': 3.0}


TrainOutput(global_step=780, training_loss=0.21081879689143254, metrics={'train_runtime': 45.0848, 'train_samples_per_second': 276.279, 'train_steps_per_second': 17.301, 'total_flos': 215919790006176.0, 'train_loss': 0.21081879689143254, 'epoch': 3.0})

# Evaluation

In [22]:
results = trainer.evaluate()
print(results)

100%|██████████| 87/87 [00:01<00:00, 77.71it/s]

{'eval_loss': 0.4941558539867401, 'eval_runtime': 1.1302, 'eval_samples_per_second': 1224.545, 'eval_steps_per_second': 76.976, 'epoch': 3.0}


In [36]:

# Get predictions for test set
predictions = trainer.predict(test_dataset)
# predicted label
preds = predictions.predictions.argmax(-1)
# true labels
labels = predictions.label_ids

# Calculate metrics
accuracy = accuracy_score(y_true=labels, y_pred=preds)
precision, recall, f1, _ = precision_recall_fscore_support(y_true=labels, y_pred=preds, average='binary')

evaluation_metrics = [
["Accuracy", accuracy],
["Precision", precision],
["Recall", recall],
["F1 Score", f1],
]
display(pd.DataFrame(evaluation_metrics, columns=["metric", "value"]))


100%|██████████| 87/87 [00:01<00:00, 79.57it/s]


,metric,value
0,Accuracy,0.895954
1,Precision,0.888430
2,Recall,0.911017
3,F1 Score,0.899582
